# 3D reconstruction for the three views setting (Person 1)

In this notebook there are done the experiments explained in the document related with the three views settings, for the first person, i.e the one placed at the left of the HC view. We will first detect and remove the outliers, then compute the baseline error and finally compute the 3D space using the differents methods described. We are going to use segments of 10 frames of different videos, which we have previously annotated the ground truth. We will do first th experiments for the body, and then for the hand.

First of all, we are going to take the frames we need for the experiment. For now, it is just going to be 10 consecutive frames from three different cameras, HC, N1 and Z1. Point1 is going to be associated with view HC, point2 to N1 and point3 to Z1. You have to comment or uncomment depending on the subjects you want to make the experiments on. The first cell is for the experiments of the body keypoints, the second cell for the hands.

In [1]:
import json

number_keypoints = 25

point1 = []
gt1 = []
for i in range (10):
# Subject 1
    with open('frames_experiments/subject1/HC/HC_0000000{}_keypoints.json'.format(11418+i), 'r') as myfile:
        data=myfile.read()
    with open('frames_experiments/subject1/HC/HC_0000000{}_keypoints_GT.json'.format(11418+i), 'r') as mygt:
        gt=mygt.read()
# Subject 3
#     with open('frames_experiments/subject3/HC/HC_0000000{}_keypoints.json'.format(32415+i), 'r') as myfile:
#         data=myfile.read()
#     with open('frames_experiments/subject3/HC/HC_0000000{}_keypoints_GT.json'.format(32415+i), 'r') as mygt:
#         gt=mygt.read()
        
    obj = json.loads(data)
    objgt = json.loads(gt)
    people = obj['people']
    peoplegt = objgt['people']
    if people[0]['pose_keypoints_2d'][0] < 640:
        p1 = people[0]['pose_keypoints_2d']
        g1 = peoplegt[0]['pose_keypoints_2d']
    else:
        p1 = people[1]['pose_keypoints_2d']
        g1 = peoplegt[1]['pose_keypoints_2d']
    p1 = [float(elem) for elem in p1]
    g1 = [float(elem) for elem in g1]
    point1.append(p1)
    gt1.append(g1)
    

point2 = []
gt2 = []
for i in range (10):
# Subject 1
    with open('frames_experiments/subject1/N1/N1_0000000{}_keypoints.json'.format(11418+i), 'r') as myfile:
        data=myfile.read()
    with open('frames_experiments/subject1/N1/N1_0000000{}_keypoints_GT.json'.format(11418+i), 'r') as mygt:
        gt=mygt.read()
# Subject 3
#     with open('frames_experiments/subject3/N1/N1_0000000{}_keypoints.json'.format(32415+i), 'r') as myfile:
#         data=myfile.read()
#     with open('frames_experiments/subject3/N1/N1_0000000{}_keypoints_GT.json'.format(32415+i), 'r') as mygt:
#         gt=mygt.read()
        
    obj = json.loads(data)
    objgt = json.loads(gt)
    people = obj['people']
    peoplegt = objgt['people']
    p2 = [float(elem) for elem in people[0]['pose_keypoints_2d']]
    g2 = [float(elem) for elem in peoplegt[0]['pose_keypoints_2d']]
    point2.append(p2)
    gt2.append(g2)

    
point3 = []
gt3 = []
for i in range (10):
# Subject 1
    with open('frames_experiments/subject1/Z1/Z1_0000000{}_keypoints.json'.format(11308+i), 'r') as myfile:
        data=myfile.read()
    with open('frames_experiments/subject1/Z1/Z1_0000000{}_keypoints_GT.json'.format(11308+i), 'r') as mygt:
        gt=mygt.read()
# Subject 3
#     with open('frames_experiments/subject3/Z1/Z1_0000000{}_keypoints.json'.format(31645+i), 'r') as myfile:
#         data=myfile.read()
#     with open('frames_experiments/subject3/Z1/Z1_0000000{}_keypoints_GT.json'.format(31645+i), 'r') as mygt:
#         gt=mygt.read()
        
    obj = json.loads(data)
    objgt = json.loads(gt)
    people = obj['people']
    peoplegt = objgt['people']
    p3 = [float(elem) for elem in people[0]['pose_keypoints_2d']]
    g3 = [float(elem) for elem in peoplegt[0]['pose_keypoints_2d']]
    point3.append(p3)
    gt3.append(g3)
    

In [2]:
import json

number_keypoints = 42

point1 = []
gt1 = []
for i in range (10):
    with open('frames_experiments/subject1/HC/HC_00000000{}_keypoints.json'.format(2290+i), 'r') as myfile:
        data=myfile.read()
    with open('frames_experiments/subject1/HC/HC_00000000{}_keypoints_GT.json'.format(2290+i), 'r') as mygt:
        gt=mygt.read()
    obj = json.loads(data)
    objgt = json.loads(gt)
    people = obj['people']
    peoplegt = objgt['people']
    if people[0]['pose_keypoints_2d'][0] < 640:
        p1_l = people[0]['hand_left_keypoints_2d']
        g1_l = peoplegt[0]['hand_left_keypoints_2d']
        p1_r = people[0]['hand_right_keypoints_2d']
        g1_r = peoplegt[0]['hand_right_keypoints_2d']
    else:
        p1_l = people[1]['hand_left_keypoints_2d']
        g1_l = peoplegt[1]['hand_left_keypoints_2d']
        p1_r = people[1]['hand_right_keypoints_2d']
        g1_r = peoplegt[1]['hand_right_keypoints_2d']
    p1_l = [float(elem) for elem in p1_l]
    g1_l = [float(elem) for elem in g1_l]
    p1_r = [float(elem) for elem in p1_r]
    g1_r = [float(elem) for elem in g1_r]
    point1.append(p1_l + p1_r)
    gt1.append(g1_l + g1_r)
    
for k in range (10):
    for i in range (0,len(point1[k]),3):
        if point1[k][i+2] < 0.1:
            point1[k][i] = 0.0
            point1[k][i+1] = 0.0
            gt1[k][i] = 0.0
            gt1[k][i+1] = 0.0

            
point2 = []
gt2 = []
for i in range (10):
    with open('frames_experiments/subject1/N1/N1_00000000{}_keypoints.json'.format(2290+i), 'r') as myfile:
        data=myfile.read()
    with open('frames_experiments/subject1/N1/N1_00000000{}_keypoints_GT.json'.format(2290+i), 'r') as mygt:
        gt=mygt.read()
    obj = json.loads(data)
    objgt = json.loads(gt)
    people = obj['people']
    peoplegt = objgt['people']
    p2_l = [float(elem) for elem in people[0]['hand_left_keypoints_2d']]
    g2_l = [float(elem) for elem in peoplegt[0]['hand_left_keypoints_2d']]
    p2_r = [float(elem) for elem in people[0]['hand_right_keypoints_2d']]
    g2_r = [float(elem) for elem in peoplegt[0]['hand_right_keypoints_2d']]
    point2.append(p2_l + p2_r)
    gt2.append(g2_l + g2_r)
    
for k in range (10):
    for i in range (0,len(point2[k]),3):
        if point2[k][i+2] < 0.1:
            point2[k][i] = 0.0
            point2[k][i+1] = 0.0
            gt2[k][i] = 0.0
            gt2[k][i+1] = 0.0

    
point3 = []
gt3 = []
for i in range (10):
    with open('frames_experiments/subject1/Z1/Z1_00000000{}_keypoints.json'.format(2267+i), 'r') as myfile:
        data=myfile.read()
    with open('frames_experiments/subject1/Z1/Z1_00000000{}_keypoints_GT.json'.format(2267+i), 'r') as mygt:
        gt=mygt.read()
    obj = json.loads(data)
    objgt = json.loads(gt)
    people = obj['people']
    peoplegt = objgt['people']
    p3_l = [float(elem) for elem in people[0]['hand_left_keypoints_2d']]
    g3_l = [float(elem) for elem in peoplegt[0]['hand_left_keypoints_2d']]
    p3_r = [float(elem) for elem in people[0]['hand_right_keypoints_2d']]
    g3_r = [float(elem) for elem in peoplegt[0]['hand_right_keypoints_2d']]
    point3.append(p3_l + p3_r)
    gt3.append(g3_l + g3_r)

for k in range (10):
    for i in range (0,len(point3[k]),3):
        if point3[k][i+2] < 0.1:
            point3[k][i] = 0.0
            point3[k][i+1] = 0.0
            gt3[k][i] = 0.0
            gt3[k][i+1] = 0.0 


## Discard outliers

Now we will proceed to discarding the outliers. For that, as we need to compute the 3D projection, we need already the matrices of intrinsic and extrinsic parameters, as well as the distortion coefficients. First of all we define some functions, such as one that computes the pixel error, and two others that commpute the multiplication between extrinsic matrices and the inverse of them. We will need this to compute all the matrices of extrinsic parameters, since as we have explained in the document, we don't have directly them all.

In [2]:
import cv2
import numpy as np

In [3]:
def error (reproj, orig):
    diff1 = abs(orig[0][0][0]-reproj[0][0][0])
    diff2 = abs(orig[0][0][1]-reproj[0][0][1])
    return [diff1, diff2]

def relative_error (reproj, orig):
    diff1 = abs(orig[0][0][0]-reproj[0][0][0])
    diff2 = abs(orig[0][0][1]-reproj[0][0][1])
    return (diff1*100/1280, diff2*100/720)

def extrinsic_inverse(M):
    N = np.identity(4)
    N[:3,:4] = M
    return np.linalg.inv(N)[:3,:4]

def extrinsic_mult(A,B):
    M = np.identity(4)
    N = np.identity(4)
    M[:3,:4] = A
    N[:3,:4] = B
    return np.dot(M,N)[:3,:4]


The intrinsic and extrinsic matrices are entered manually from the yml files of calibration. Execute the following cell for Subject 1

In [4]:
# Intrinsic matrices
I_HC = np.array([[1561.0879250519019, 0., 696.26643757458214], 
                 [0., 1559.8875934756718, 364.00150809603042], 
                 [0., 0., 1.]])
I_N1 = np.array([[1.5302606428953902e+03, 0., 6.3704463363410275e+02], 
                 [0., 1.5363325136178830e+03, 4.0223336660626796e+02], 
                 [0., 0., 1.]])
I_Z1 = np.array([[1.7186882095790779e+03, 0., 5.9860544121349926e+02], 
                 [0., 1.5403542389142547e+03, 3.8155238056391255e+02], 
                 [0., 0., 1.]])


# Extrinsic matrices
E_HC = np.array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.]])
E_id = E_HC # matrix identity

E_N2HC = np.array([[4.7071373825592244e-01, -3.1599447001591768e-01, 8.2375728921600100e-01, -1.0304478556951631e+03],
                    [2.8802340986999736e-01, 9.3754663626875434e-01, 1.9506106784287736e-01, -3.0531267263949093e+02],
                    [-8.3394909436007802e-01, 1.4544345891274529e-01, 5.3232988670137815e-01, 9.1388342483064150e+02]])
E_N1N2 = np.array([[-5.4986998367568851e-01, 3.0581005268310585e-01, -7.7725363474894993e-01, 8.3663027029970908e+02],
                   [-2.2725644654053018e-01, 8.4068086603861980e-01, 4.9153859358378738e-01, -6.3514254777430710e+02],
                   [8.0373970199204992e-01, 4.4691821752364558e-01, -3.9276786819598442e-01, 1.8039039431233032e+03]])
E_N1Z1 = np.array([[9.7907944983707595e-01, 7.5806963742383632e-02, -1.8882991064682936e-01, 4.7378144665229661e+02], 
                   [-4.3073408183321484e-02, 9.8419391226508868e-01, 1.7177608846347608e-01, -3.6645021346744181e+02],
                   [1.9886707222212935e-01, -1.6004889036947406e-01, 9.6686929844591629e-01, 2.8227302985118314e+02]])

E_HCN2 = extrinsic_inverse(E_N2HC)
E_N2N1 = extrinsic_inverse(E_N1N2)

E_HCN1 = extrinsic_mult(E_N2N1, E_HCN2)
E_HCZ1 = extrinsic_mult(E_N1Z1, E_HCN1)


# Distortion parameters
k_HC = np.array([-3.6946481172355405e-01, 1.7682498387118720e-01, 0., 0., 0.])
k_N1 = np.array([-3.5111725250493975e-01, 1.5902320213719132e-01, 0., 0., 0.])
k_Z1 = np.array([-3.5841115333091467e-01, 1.1657468374802429e-01, 0., 0., 0.])

Execute following cell for Subject 3

In [5]:
# Intrinsic matrices
I_HC = np.array([[1.5499983892925129e+03, 0., 6.7370178950483637e+02], 
                 [0., 1.5555084214552871e+03, 3.5903923720178398e+02], 
                 [0., 0., 1.]])
I_N1 = np.array([[1.5293912158286053e+03, 0., 6.5418998630403360e+02], 
                 [0., 1.5343331678263205e+03, 3.9346658022445314e+02], 
                 [0., 0., 1.]])
I_Z1 = np.array([[1.7269416360213065e+03, 0., 6.5402479790662915e+02], 
                 [0., 1.5504713027220994e+03, 3.9660442330342642e+02], 
                 [0., 0., 1.]])


# Extrinsic matrices
E_HC = np.array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.]])
E_id = E_HC # matrix identity

E_N2HC = np.array([[4.1228656953386672e-01, -3.2483381861465577e-01, 8.5117728756482769e-01, -9.9037881350817702e+02], 
                   [2.8128111321437588e-01, 9.3401885188095635e-01, 2.2020381395394090e-01, -3.1043993888490388e+02],
                   [-8.6654527863861663e-01, 1.4863301993568356e-01, 4.7645304643156722e-01, 9.3569732917438898e+02]])
E_N1N2 = np.array([[-5.8806368848874491e-01, 2.6854418111967210e-01, -7.6293192426833034e-01, 7.1720040763760062e+02], 
                   [-2.4314777291204553e-01, 8.4094246273336348e-01, 4.8342003982029774e-01, -6.4274374358329874e+02],
                   [7.7140149002249458e-01, 4.6978696997544606e-01, -4.2923180687405327e-01, 1.8581775152396260e+03]])
E_N1Z1 = np.array([[9.7246776211223829e-01, 7.4575985473482945e-02, -2.2078241334643017e-01, 4.8167447731443514e+02], 
                   [-4.2143550637449734e-02, 9.8807050286782827e-01, 1.4812360548604550e-01, -3.4560800428335369e+02],
                   [2.2919505403058721e-01, -1.3474087632628709e-01, 9.6400960755313037e-01, 3.1835695266983555e+02]])

E_HCN2 = extrinsic_inverse(E_N2HC)
E_N2N1 = extrinsic_inverse(E_N1N2)

E_HCN1 = extrinsic_mult(E_N2N1, E_HCN2)
E_HCZ1 = extrinsic_mult(E_N1Z1, E_HCN1)


# Distortion parameters
k_HC = np.array([-3.5867296194490861e-01, 1.5624702217456826e-01, 0., 0., 0.])
k_N1 = np.array([-3.5769234546998718e-01, 1.7106341625672861e-01, 0., 0., 0.])
k_Z1 = np.array([-3.6090208297293613e-01, 9.0434516483797828e-02, 0., 0., 0.])

The next cell is the function that discards the outliers. As we have explained, it project the points to 3D for every pair of views and reproject to the 2D views and compute the error. If the error is bigger than the experimentally treshold set to 15, we remove them, i.e set to zero.

In [6]:
def discard_outliers(point1, point2, point3, I1, I2, I3, E1, E2, E3, k1, k2, k3, length, t):
    # We will set values to zero to the outliers
    new_point1 = point1.copy()
    new_point2 = point2.copy()
    new_point3 = point3.copy()
    
    for i in range(0,length,3):
        
        
        if point1[i] != 0 or point2[i] != 0 or point3[i] != 0:
            
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p2 = np.array([[[point2[i], point2[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            
            projPoints1 = cv2.undistortPoints(p1, I1, k1)
            projPoints2 = cv2.undistortPoints(p2, I2, k2)
            projPoints3 = cv2.undistortPoints(p3, I3, k3)
            
            world_coord_12 = cv2.triangulatePoints(E1, E2, projPoints1, projPoints2) 
            world_coord_norm_12 = (world_coord_12[:-1]/world_coord_12[3]).reshape(1,1,3)
            world_coord_13 = cv2.triangulatePoints(E1, E3, projPoints1, projPoints3) 
            world_coord_norm_13 = (world_coord_13[:-1]/world_coord_13[3]).reshape(1,1,3)     
            world_coord_23 = cv2.triangulatePoints(E2, E3, projPoints2, projPoints3) 
            world_coord_norm_23 = (world_coord_23[:-1]/world_coord_23[3]).reshape(1,1,3)
            
            # This is for reprojection.
            R1 = E1[:3,:3]
            T1 = E1[:,3]
            R2 = E2[:3,:3]
            T2 = E2[:,3]
            R3 = E3[:3,:3]
            T3 = E3[:,3]
            
            # This array is to check which view is wrong, if there is
            error_view = np.array([0,0,0])
            
            reproj_2d_12_1 = cv2.projectPoints(world_coord_norm_12, R1, T1, I1, k1)[0]
            reproj_2d_12_2 = cv2.projectPoints(world_coord_norm_12, R2, T2, I2, k2)[0]
            if (error(reproj_2d_12_1, p1)[0] > t or error(reproj_2d_12_1, p1)[1] > t) and (error(reproj_2d_12_2, p2)[0] > t or error(reproj_2d_12_2, p2)[1] > t):
                error_view[0] = 1
                #This commented lines are to print the error and see it quantitatively
#                 print("\n Error HC: ", error(reproj_2d_12_1, p1))
#                 print(i, p1)
#                 print("\n Error N1: ", error(reproj_2d_12_2, p2))
#                 print(i, p2)

            
            reproj_2d_13_1 = cv2.projectPoints(world_coord_norm_13, R1, T1, I1, k1)[0]
            reproj_2d_13_2 = cv2.projectPoints(world_coord_norm_13, R3, T3, I3, k3)[0]
            if (error(reproj_2d_13_1, p1)[0] > t or error(reproj_2d_13_1, p1)[1] > t) and (error(reproj_2d_13_2, p3)[0] > t or error(reproj_2d_13_2, p3)[1] > t):
                error_view[1] = 1
#                 print("\n Error HC: ", error(reproj_2d_13_1, p1))
#                 print(i, p1)
#                 print("\n Error Z1: ", error(reproj_2d_13_2, p3))
#                 print(i, p3)
            
            reproj_2d_23_1 = cv2.projectPoints(world_coord_norm_23, R2, T2, I2, k2)[0]
            reproj_2d_23_2 = cv2.projectPoints(world_coord_norm_23, R3, T3, I3, k3)[0]
            if (error(reproj_2d_23_1, p2)[0] > t or error(reproj_2d_23_1, p2)[1] > t) and (error(reproj_2d_23_2, p3)[0] > t or error(reproj_2d_23_2, p3)[1] > t):
                error_view[2] = 1
#                 print("\n Error N1: ", error(reproj_2d_23_1, p2))
#                 print(i, p2)
#                 print("\n Error Z1: ", error(reproj_2d_23_2, p3))
#                 print(i, p3)
             

            if error_view.any(): # we enter here if error_view != (0,0,0). I will just consider an outlier if there are two 1's
                # I put three different if to set 0 the cases when just one point over the three views is detected. I put that
                # the points to be set to zero need to be different from zero because it is stupid to set 0 something that is
                # already 0
                if error_view[0] and error_view[1] and point1[i] != 0:
                    print("Error in view HC of the point {}".format(i/3))
                    new_point1[i] = 0.0
                    new_point1[i+1] = 0.0
                    new_point1[i+2] = 0.0
                if error_view[0] and error_view[2] and point2[i] != 0:
                    print("Error in view N1 of the point {}".format(i/3))
                    new_point2[i] = 0.0
                    new_point2[i+1] = 0.0
                    new_point2[i+2]= 0.0
                if error_view[1] and error_view[2] and point3[i] != 0:
                    print("Error in view Z1 of the point {}".format(i/3))
                    new_point3[i] = 0.0
                    new_point3[i+1] = 0.0
                    new_point3[i+2] = 0.0
                    
                    
    return new_point1, new_point2, new_point3

In [12]:
# We create three list of list to put the new_points
new_point1 = []
new_point2 = []
new_point3 = []
threshold = 15
for i in range(10):
        print("Frame number {}".format(i+1))
        new_1, new_2, new_3=discard_outliers(point1[i], point2[i], point3[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(point1[i]), threshold)
        new_point1.append(new_1)
        new_point2.append(new_2)
        new_point3.append(new_3)
        print("\n")

Frame number 1
Error in view HC of the point 8.0
Error in view Z1 of the point 8.0
Error in view HC of the point 9.0
Error in view Z1 of the point 9.0
Error in view HC of the point 12.0
Error in view Z1 of the point 12.0


Frame number 2
Error in view HC of the point 8.0
Error in view Z1 of the point 8.0
Error in view HC of the point 9.0
Error in view Z1 of the point 9.0
Error in view HC of the point 12.0
Error in view Z1 of the point 12.0


Frame number 3
Error in view Z1 of the point 4.0
Error in view HC of the point 8.0
Error in view Z1 of the point 8.0
Error in view HC of the point 9.0
Error in view Z1 of the point 9.0
Error in view HC of the point 12.0
Error in view Z1 of the point 12.0


Frame number 4
Error in view HC of the point 8.0
Error in view Z1 of the point 8.0
Error in view HC of the point 9.0
Error in view Z1 of the point 9.0
Error in view HC of the point 12.0
Error in view Z1 of the point 12.0


Frame number 5
Error in view HC of the point 8.0
Error in view Z1 of the p

Usually, in this cases that there is an error in two different views, it is because for one of the three views the point wasn't detected, and then the 3D projection between the other two views wasn't accurate, meaning that one of those was an outlier. Since it doesn't make sense to keep just one point (we can't project to 3D with just one point), we set both to zero.

## 3D projection and 2D reprojection

We will proceed now to the 3D projection and 2D reprojection using the different methods. First we define a function that projects the 2D points of two different calibrated views into a 3D point. Also the function that reprojects a 3D point into the three views, compares it with the ground truth (if exists) and get the pixel error. Also some functions that are useful to make operations between the errors and print them.

In [14]:
def reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2):
    projPoints1 = cv2.undistortPoints(p1, I1, k1)
    projPoints2 = cv2.undistortPoints(p2, I2, k2)

    world_coord = cv2.triangulatePoints(E1, E2, projPoints1, projPoints2) 
    world_coord_norm = world_coord/world_coord[3]

    new_world_coord = (world_coord[:-1]/world_coord[3]).reshape(1,1,3)
    return new_world_coord[0]

In [15]:
def reprojection_2D(ground_truth1, ground_truth2, ground_truth3, total_points_3D, I1, I2, I3, E1, E2, E3, k1, k2, k3, length):
    
    error1 = []
    error2 = []
    error3 = []
    
    
    for i in range(0,length,3):

        R1 = E1[:3,:3]
        T1 = E1[:,3]
        R2 = E2[:3,:3]
        T2 = E2[:,3]
        R3 = E3[:3,:3]
        T3 = E3[:,3]

        gt1 = np.array([[[ground_truth1[i], ground_truth1[i+1]]]])
        gt2 = np.array([[[ground_truth2[i], ground_truth2[i+1]]]])
        gt3 = np.array([[[ground_truth3[i], ground_truth3[i+1]]]])
        
        # the length of the total_points_3D is the total length divided by three
        world_coord = total_points_3D[int(i/3)].reshape(1,1,3)
        
        if world_coord.any() != 0: # we just want to reproject if the point 3D is different from (0,0,0). That means that at 
                                   # least two different 2D points are available.
        
            if gt1.any() != 0: 
                reproj_2d_1 = cv2.projectPoints(world_coord, R1, T1, I1, k1)[0]
#                 print(reproj_2d_1, gt1)
                error1.append(error(reproj_2d_1, gt1))
            else:
                error1.append([0,0])

            if gt2.any() != 0:
                reproj_2d_2 = cv2.projectPoints(world_coord, R2, T2, I2, k2)[0]
#                 print(reproj_2d_2, gt2)
                error2.append(error(reproj_2d_2, gt2))
            else:
                error2.append([0,0])

            if gt3.any() != 0:
                reproj_2d_3 = cv2.projectPoints(world_coord, R3, T3, I3, k3)[0]
#                 print(reproj_2d_3, gt3)
                error3.append(error(reproj_2d_3, gt3))
            else:
                error3.append([0,0])
    
        else:
            error1.append([0,0])
            error2.append([0,0])
            error3.append([0,0])
                

#     return (np.mean(error1), np.mean(error2), np.mean(error3)), number_points
    return error1, error2, error3

In [23]:
def sum_error(x,y,c):
    res = np.zeros((number_keypoints,2))
    for i in range (len(x)):
        if y[i] != [0,0]:
            c[i] += 1
        for j in range (len(x[i])):
            res[i][j] = x[i][j]+y[i][j]
    return res, c

def dividing_error(x,c):
    for i in range (len(x)):
        if c[i] != 0:
            x[i] = x[i]/c[i]
    return x

def error_printing(x,y,z):
    mean = []
    for i in range (len(x)):
        print("Point {}: [{:.2f},{:.2f}] ; [{:.2f},{:.2f}] ; [{:.2f},{:.2f}] ---- Mean: {:.2f}".format(i, x[i][0], x[i][1], y[i][0], y[i][1], z[i][0], z[i][1], mean_errors(x[i], y[i], z[i])))
#         print("{} & [{:.2f},{:.2f}] & [{:.2f},{:.2f}] & [{:.2f},{:.2f}] & {:.2f} \\\\".format(i, x[i][0], x[i][1], y[i][0], y[i][1], z[i][0], z[i][1], mean_errors(x[i], y[i], z[i])))
        mean.append((x[i], y[i], z[i], mean_errors(x[i], y[i], z[i])))
    return mean
        
def mean_errors(a,b,c):
    count = 0
    if a.any() != 0:
        count += 1
    if b.any() != 0:
        count += 1
    if c.any() != 0:
        count += 1
    if count != 0:
        return (np.mean(a)+np.mean(b)+np.mean(c))/count
    else: 
        return 0

        

### Baseline error

We are going to compute now the baseline error, which is going to be the mean of the error for every pair of views, as explained in the document. This is going to be as reference point to compare with all the methods we propose. 

In [24]:
def baseline_error(point1, point2, point3, I1, I2, I3, E1, E2, E3, k1, k2, k3, length):
    points_3D_12 = []
    points_3D_13 = []
    points_3D_23 = []
    
    for i in range(0,length,3):
        
        c = 0 # to count the number of pairs of views we have available.

        if point1[i] != 0 and point2[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p2 = np.array([[[point2[i], point2[i+1]]]])
            point_3D_12 = reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2)
            points_3D_12.extend(point_3D_12)
        else:
            points_3D_12.extend(np.array([[0,0,0]]))
            
        if point1[i] != 0 and point3[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            point_3D_13 = reconstruction_3D(p1, p3, I1, I3, E1, E3, k1, k3)
            points_3D_13.extend(point_3D_13)
        else:
            points_3D_13.extend(np.array([[0,0,0]]))
            
        if point2[i] != 0 and point3[i] != 0:
            p2 = np.array([[[point2[i], point2[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            point_3D_23 = reconstruction_3D(p2, p3, I2, I3, E2, E3, k2, k3)
            points_3D_23.extend(point_3D_23)
        else:
            points_3D_23.extend(np.array([[0,0,0]]))
            
        
    return points_3D_12, points_3D_13, points_3D_23

In [25]:
points_3D_12 = []
points_3D_13 = []
points_3D_23 = []
for i in range(10):
    points_12, points_13, points_23 = baseline_error(new_point1[i], new_point2[i], new_point3[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(point1[i]))
    points_3D_12.append(points_12)
    points_3D_13.append(points_13)
    points_3D_23.append(points_23)

In [26]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], points_3D_12[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("HC-N1 view")
mean_error_HCN1 = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))

HC-N1 view
Point 0: [0.33,1.62] ; [0.12,1.95] ; [7.55,13.49] ---- Mean: 4.17
Point 1: [1.28,5.53] ; [0.47,6.46] ; [13.00,11.35] ---- Mean: 6.35
Point 2: [0.54,2.36] ; [0.18,2.58] ; [8.79,6.67] ---- Mean: 3.52
Point 3: [4.04,11.82] ; [1.08,12.64] ; [11.05,35.87] ---- Mean: 12.75
Point 4: [4.52,11.63] ; [1.16,12.93] ; [10.55,21.09] ---- Mean: 10.31
Point 5: [3.12,13.72] ; [1.23,17.11] ; [7.66,9.73] ---- Mean: 8.76
Point 6: [1.14,3.47] ; [0.51,4.44] ; [15.88,19.23] ---- Mean: 7.44
Point 7: [1.15,3.05] ; [0.48,3.93] ; [27.94,13.76] ---- Mean: 8.39
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- M

Here we see the error obtained with the projection 3D of the views HC-N1. We then reproject and compare with the three views. We see that the errors are quite bigger in the Z1 view than in the others. This is why we aren't taking this view into account. The number of points express the points that had ground truth for every of the views, from 0 to 24.

In [27]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], points_3D_13[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("HC-Z1 view")
mean_error_HCZ1 = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))

HC-Z1 view
Point 0: [1.17,7.90] ; [6.09,4.55] ; [2.35,7.90] ---- Mean: 4.99
Point 1: [1.09,9.55] ; [11.93,3.55] ; [2.98,9.42] ---- Mean: 6.42
Point 2: [0.33,3.27] ; [10.55,7.72] ; [1.00,3.08] ---- Mean: 4.32
Point 3: [0.37,14.55] ; [18.00,9.74] ; [20.89,29.75] ---- Mean: 15.55
Point 4: [0.28,14.01] ; [25.04,9.28] ; [5.01,13.69] ---- Mean: 11.22
Point 5: [1.52,11.80] ; [1.53,20.93] ; [3.71,12.17] ---- Mean: 8.61
Point 6: [0.74,8.47] ; [18.83,17.27] ; [3.07,9.03] ---- Mean: 9.57
Point 7: [0.63,8.81] ; [33.67,8.47] ; [3.42,9.57] ---- Mean: 10.76
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [20.28,12.92] ; [0.00,0.00] ; [4.96,12.51] ---- Mean: 12.67
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---

In [28]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], points_3D_23[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("N1-Z1 view")
mean_error_N1Z1 = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))

N1-Z1 view
Point 0: [23.54,10.80] ; [1.78,2.88] ; [1.88,2.41] ---- Mean: 7.22
Point 1: [25.45,16.90] ; [0.69,1.09] ; [0.74,0.93] ---- Mean: 7.63
Point 2: [26.01,2.63] ; [0.84,1.37] ; [0.92,1.20] ---- Mean: 5.50
Point 3: [51.35,27.20] ; [2.66,4.12] ; [17.73,19.69] ---- Mean: 20.46
Point 4: [17.14,25.52] ; [4.43,6.65] ; [5.15,5.99] ---- Mean: 10.81
Point 5: [3.02,26.14] ; [2.58,4.05] ; [2.77,3.35] ---- Mean: 6.99
Point 6: [47.19,10.61] ; [2.62,3.76] ; [2.93,3.08] ---- Mean: 11.70
Point 7: [49.89,15.32] ; [1.81,2.58] ; [2.00,2.13] ---- Mean: 12.29
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- 

In [30]:
# Compute and print the baseline error
print("Baseline error")
final_mean = 0
c_final = 0
for i in range (number_keypoints):
    c = np.zeros((3))
    c_total = 0
    aux = np.zeros((3,2))
    for j in range (3):
        if mean_error_HCN1[i][j].any() != 0:
            c[j] += 1
        if mean_error_HCZ1[i][j].any() != 0:
            c[j] += 1
        if mean_error_N1Z1[i][j].any() != 0:
            c[j] += 1
        if c[j] != 0:
            aux[j] = (mean_error_HCN1[i][j] + mean_error_HCZ1[i][j] + mean_error_N1Z1[i][j])/c[j]
    if mean_error_HCN1[i][3] != 0:
        c_total += 1
    if mean_error_HCZ1[i][3] != 0:
        c_total += 1
    if mean_error_N1Z1[i][3] != 0:
        c_total += 1
    if c_total != 0:
        total_mean = (mean_error_HCN1[i][3] + mean_error_HCZ1[i][3] + mean_error_N1Z1[i][3])/c_total
        final_mean += total_mean
        c_final += 1
    else:
        total_mean = 0.0  
    print("Point {}: [{:.2f},{:.2f}] ; [{:.2f},{:.2f}] ; [{:.2f},{:.2f}] ---- Mean: {:.2f}".format(i, aux[0][0], aux[0][1], aux[1][0], aux[1][1], aux[2][0], aux[2][1], total_mean))
#     print("{} & [{:.2f},{:.2f}] & [{:.2f},{:.2f}] & [{:.2f},{:.2f}] & {:.2f} \\\\".format(i, aux[0][0], aux[0][1], aux[1][0], aux[1][1], aux[2][0], aux[2][1], total_mean))
print("\nFinal mean: {:.2f}".format(final_mean/c_final))       
            


Baseline error
Point 0: [8.35,6.77] ; [2.66,3.13] ; [3.92,7.93] ---- Mean: 5.46
Point 1: [9.27,10.66] ; [4.36,3.70] ; [5.57,7.23] ---- Mean: 6.80
Point 2: [8.96,2.75] ; [3.86,3.89] ; [3.57,3.65] ---- Mean: 4.45
Point 3: [18.59,17.86] ; [7.25,8.83] ; [16.56,28.44] ---- Mean: 16.25
Point 4: [7.31,17.05] ; [10.21,9.62] ; [6.90,13.59] ---- Mean: 10.78
Point 5: [2.55,17.22] ; [1.78,14.03] ; [4.71,8.42] ---- Mean: 8.12
Point 6: [16.36,7.52] ; [7.32,8.49] ; [7.29,10.44] ---- Mean: 9.57
Point 7: [17.22,9.06] ; [11.99,4.99] ; [11.12,8.49] ---- Mean: 10.48
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [20.28,12.92] ; [0.00,0.00] ; [4.96,12.51] ---- Mean: 12.67
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00]

When we have the errors computed for every pair of views, we do the mean between them for each keypoint. This is the baseline error. The last element of each row is the mean of all the errors (x,y) found for each view. The last value, final mean, is the mean of these last values of each row.

### Mean of the three views

The first case to do 3D reprojection, is taking the mean of the three pair of views we have, once the outliers have been discarded.

In [31]:
def mean_reconstruction_3D(point1, point2, point3, I1, I2, I3, E1, E2, E3, k1, k2, k3, length):
    mean_points_3D = []
    
    for i in range(0,length,3):
        
        c = 0 # to count the number of pairs of views we have available.
        sum_value = np.array([[0,0,0]]) # to sum the values of the views for making the mean

        if point1[i] != 0 and point2[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p2 = np.array([[[point2[i], point2[i+1]]]])
            point_3D_12 = reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2)
#             print(point_3D_12)
            c += 1
            sum_value = sum_value + point_3D_12
            
        if point1[i] != 0 and point3[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            point_3D_13 = reconstruction_3D(p1, p3, I1, I3, E1, E3, k1, k3)
#             print(point_3D_13)
            c += 1
            sum_value = sum_value + point_3D_13
            
        if point2[i] != 0 and point3[i] != 0:
            p2 = np.array([[[point2[i], point2[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            point_3D_23 = reconstruction_3D(p2, p3, I2, I3, E2, E3, k2, k3)
#             print(point_3D_23)
            c += 1
            sum_value = sum_value + point_3D_23
        
        if c != 0:
            mean = sum_value/c
        else:
            mean = sum_value
#         print(mean, '\n')
        
        # We always add the points, even if we don't go inside the if. Then we are just adding three 0's.
        mean_points_3D.extend(mean)
            
        
    return mean_points_3D

In [32]:
mean_points_3D = []
for i in range(10):
    mean_3D= mean_reconstruction_3D(new_point1[i], new_point2[i], new_point3[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(point1[i]))
    mean_points_3D.append(mean_3D)

In [33]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], mean_points_3D[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("Mean of the three views")
mean_error_mean = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
final_mean = 0
c = 0
for i in range (number_keypoints):
    if mean_error_mean[i][3] != 0:
        final_mean += mean_error_mean[i][3]
        c += 1
print("\nFinal mean: {:.2f}".format(final_mean/c))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))
        

Mean of the three views
Point 0: [8.05,6.76] ; [1.42,1.82] ; [2.70,7.97] ---- Mean: 4.79
Point 1: [8.31,10.63] ; [4.09,3.73] ; [5.60,6.68] ---- Mean: 6.51
Point 2: [8.85,1.18] ; [3.34,3.92] ; [2.98,3.67] ---- Mean: 3.99
Point 3: [17.36,17.78] ; [6.45,5.91] ; [15.05,28.55] ---- Mean: 15.18
Point 4: [6.90,16.50] ; [9.12,6.25] ; [3.54,13.90] ---- Mean: 9.37
Point 5: [0.93,17.21] ; [0.96,14.05] ; [4.71,6.19] ---- Mean: 7.34
Point 6: [15.98,5.15] ; [5.76,8.59] ; [5.45,10.55] ---- Mean: 8.58
Point 7: [16.53,7.81] ; [12.14,3.73] ; [11.20,7.22] ---- Mean: 9.77
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [20.28,12.92] ; [0.00,0.00] ; [4.96,12.51] ---- Mean: 12.67
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0.00,0.00] ; [0.00

### Remove the least confident and take the mean of the other two

We are going to remove the least confident point, and take the 3D reprojection obtained with the other two points.


In [34]:
def removing_confidence_reconstruction_3D(point1, point2, point3, I1, I2, I3, E1, E2, E3, k1, k2, k3, length):
    mean_points_3D = []
    
    for i in range(0,length,3):
        
        final_point_3D = np.array([[0,0,0]])
        confidence_1 = point1[i+2]
        confidence_2 = point2[i+2]
        confidence_3 = point3[i+2]
        
        if confidence_1 >= confidence_2:
            if confidence_2 >= confidence_3:
                p1 = np.array([[[point1[i], point1[i+1]]]])
                p2 = np.array([[[point2[i], point2[i+1]]]])
                point_3D_12 = reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2)
                if confidence_1 != 0 and confidence_2 != 0:
                    final_point_3D = point_3D_12
            else: #this comprends two different elifs, confidence_2 < confidence_3 and confidence_1 < confidence_3
                p1 = np.array([[[point1[i], point1[i+1]]]])
                p3 = np.array([[[point3[i], point3[i+1]]]])
                point_3D_13 = reconstruction_3D(p1, p3, I1, I3, E1, E3, k1, k3)
                if confidence_1 != 0 and confidence_3 != 0:
                    final_point_3D = point_3D_13

        else:
            if confidence_1 >= confidence_3:
                p1 = np.array([[[point1[i], point1[i+1]]]])
                p2 = np.array([[[point2[i], point2[i+1]]]])
                point_3D_12 = reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2)
                if confidence_1 != 0 and confidence_2 != 0:
                    final_point_3D = point_3D_12
            else: 
                p2 = np.array([[[point2[i], point2[i+1]]]])
                p3 = np.array([[[point3[i], point3[i+1]]]])
                point_3D_23 = reconstruction_3D(p2, p3, I2, I3, E2, E3, k2, k3)
                if confidence_2 != 0 and confidence_3 != 0:
                    final_point_3D = point_3D_23 

        
#         print(i/3, final_point_3D, '\n')
           
        # We always add the points, even if we don't go inside the if. Then we are just adding three 0's.
        mean_points_3D.extend(final_point_3D)
            
    return mean_points_3D

In [35]:
removing_confidence_points_3D = []
for i in range(10):
    removing_confidence_3D = removing_confidence_reconstruction_3D(new_point1[i], new_point2[i], new_point3[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(point1[i]))
    removing_confidence_points_3D.append(removing_confidence_3D)

In [36]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], removing_confidence_points_3D[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("Removing least confident and taking other two")
mean_error_confidence = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
final_mean = 0
c = 0
for i in range (number_keypoints):
    if mean_error_confidence[i][3] != 0:
        final_mean += mean_error_confidence[i][3]
        c += 1
print("\nFinal mean: {:.2f}".format(final_mean/c))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))

Removing least confident and taking other two
Point 0: [0.72,4.55] ; [2.78,3.09] ; [5.21,10.85] ---- Mean: 4.53
Point 1: [17.78,14.68] ; [4.21,1.76] ; [1.45,3.57] ---- Mean: 7.24
Point 2: [26.01,2.63] ; [0.84,1.37] ; [0.92,1.20] ---- Mean: 5.50
Point 3: [13.05,20.08] ; [6.07,10.85] ; [11.49,26.93] ---- Mean: 14.75
Point 4: [4.52,11.63] ; [1.16,12.93] ; [10.55,21.09] ---- Mean: 10.31
Point 5: [1.52,11.80] ; [1.53,20.93] ; [3.71,12.17] ---- Mean: 8.61
Point 6: [4.68,8.80] ; [17.66,16.29] ; [3.03,8.38] ---- Mean: 9.81
Point 7: [0.63,8.81] ; [33.67,8.47] ; [3.42,9.57] ---- Mean: 10.76
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [20.28,12.92] ; [0.00,0.00] ; [4.96,12.51] ---- Mean: 12.67
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.0

### Weighted mean

We are simply computing the weighted mean using as weights the confidence values.

In [37]:
def weighted_mean_reconstruction_3D(point1, point2, point3, I1, I2, I3, E1, E2, E3, k1, k2, k3, length):
    mean_points_3D = []
    
    for i in range(0,length,3):
        
        c = 0 # to count the number of pairs of views we have available.
        sum_value = np.array([[0,0,0]]) # to sum the values of the views for making the mean

        if point1[i] != 0 and point2[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p2 = np.array([[[point2[i], point2[i+1]]]])
            confidence_12 = np.mean(np.array([point1[i+2], point2[i+2]]))
            point_3D_12 = reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2)
#             print(point_3D_12)
            c += confidence_12
            sum_value = sum_value + point_3D_12*confidence_12
            
        if point1[i] != 0 and point3[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            confidence_13 = np.mean(np.array([point1[i+2], point3[i+2]]))
            point_3D_13 = reconstruction_3D(p1, p3, I1, I3, E1, E3, k1, k3)
#             print(point_3D_13)
            c += confidence_13
            sum_value = sum_value + point_3D_13*confidence_13
            
        if point2[i] != 0 and point3[i] != 0:
            p2 = np.array([[[point2[i], point2[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            confidence_23 = np.mean(np.array([point2[i+2], point3[i+2]]))
            point_3D_23 = reconstruction_3D(p2, p3, I2, I3, E2, E3, k2, k3)
#             print(point_3D_23)
            c += confidence_23
            sum_value = sum_value + point_3D_23*confidence_23
        
        if c != 0:
            mean = sum_value/c
        else:
            mean = sum_value
#         print(i/3, mean, '\n')
        
        # We always add the points, even if we don't go inside the if. Then we are just adding three 0's.
        mean_points_3D.extend(mean)
            
        
    return mean_points_3D

In [38]:
weighted_mean_points_3D = []
for i in range(10):
    weighted_mean_3D = weighted_mean_reconstruction_3D(new_point1[i], new_point2[i], new_point3[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(point1[i]))
    weighted_mean_points_3D.append(weighted_mean_3D)

In [39]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], weighted_mean_points_3D[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("Weighted mean of the three views")
mean_error_weighted = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
final_mean = 0
c = 0
for i in range (number_keypoints):
    if mean_error_weighted[i][3] != 0:
        final_mean += mean_error_weighted[i][3]
        c += 1
print("\nFinal mean: {:.2f}".format(final_mean/c))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))

Weighted mean of the three views
Point 0: [8.00,6.75] ; [1.43,1.82] ; [2.72,7.99] ---- Mean: 4.78
Point 1: [8.40,10.68] ; [4.13,3.70] ; [5.54,6.63] ---- Mean: 6.51
Point 2: [9.02,1.20] ; [3.32,3.90] ; [2.92,3.64] ---- Mean: 4.00
Point 3: [17.02,17.82] ; [6.49,6.01] ; [14.90,28.52] ---- Mean: 15.13
Point 4: [6.61,16.19] ; [8.50,6.63] ; [4.14,14.39] ---- Mean: 9.41
Point 5: [0.91,17.07] ; [0.95,14.22] ; [4.71,6.35] ---- Mean: 7.37
Point 6: [15.87,5.18] ; [5.86,8.66] ; [5.43,10.54] ---- Mean: 8.59
Point 7: [14.93,8.12] ; [14.86,4.31] ; [10.02,7.33] ---- Mean: 9.93
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [20.28,12.92] ; [0.00,0.00] ; [4.96,12.51] ---- Mean: 12.67
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0.00,0.00

### Removing the highest distance and take the mean of the other two

We remove the highest distance compared doing the mean of the other three pair of views, remove it and compute the mean of the two remaining values.

In [40]:
def removing_distance_reconstruction_3D(point1, point2, point3, I1, I2, I3, E1, E2, E3, k1, k2, k3, length):
    mean_points_3D = []
    
    for i in range(0,length,3):
        
        sum_value = np.array([[0,0,0]]) # to sum the values of the views for making the mean
        c = 0

        if point1[i] != 0 and point2[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p2 = np.array([[[point2[i], point2[i+1]]]])
            point_3D_12 = reconstruction_3D(p1, p2, I1, I2, E1, E2, k1, k2)
#             print(point_3D_12)
            c += 1
            sum_value = sum_value + point_3D_12

        if point1[i] != 0 and point3[i] != 0:
            p1 = np.array([[[point1[i], point1[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            point_3D_13 = reconstruction_3D(p1, p3, I1, I3, E1, E3, k1, k3)
#             print(point_3D_13)
            c += 1
            sum_value = sum_value + point_3D_13
            
        if point2[i] != 0 and point3[i] != 0:
            p2 = np.array([[[point2[i], point2[i+1]]]])
            p3 = np.array([[[point3[i], point3[i+1]]]])
            point_3D_23 = reconstruction_3D(p2, p3, I2, I3, E2, E3, k2, k3)
#             print(point_3D_23)
            c += 1
            sum_value = sum_value + point_3D_23
        
        need_to_operate = 0 # This variable is just to know if we have to operate with distances or not
        if c == 3:
            mean = sum_value/c
            need_to_operate = 1
        else:
            final_mean = sum_value

        sum_value = np.array([[0,0,0]])
        if need_to_operate == 1:
            distance_12 = abs(np.mean(abs(point_3D_12) - abs(mean)))
            distance_13 = abs(np.mean(abs(point_3D_13) - abs(mean)))
            distance_23 = abs(np.mean(abs(point_3D_23) - abs(mean)))
#             print(distance_12, distance_13, distance_23)
        
            if distance_12 < distance_13:
                if distance_13 < distance_23:
                    sum_value = sum_value + point_3D_12 + point_3D_13
                else: #this comprends two different elifs, distance_13 >= distance_23 and distance_12 >= distance_23
                    sum_value = sum_value + point_3D_12 + point_3D_23
            else:
                if distance_12 < distance_23:
                    sum_value = sum_value + point_3D_12 + point_3D_13
                else: 
                    sum_value = sum_value + point_3D_13 + point_3D_23 
                    
            final_mean = sum_value/2
        
#         print(i/3, final_mean, '\n')
        
        # We always add the points, even if we don't go inside the if. Then we are just adding three 0's.
        mean_points_3D.extend(final_mean)
            
    return mean_points_3D

In [41]:
removing_distance_points_3D = []
for i in range(10):
    removing_distance_3D = removing_distance_reconstruction_3D(new_point1[i], new_point2[i], new_point3[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(point1[i]))
    removing_distance_points_3D.append(removing_distance_3D)

In [42]:
sum_HC = [(0,0)]*number_keypoints
sum_N1 = [(0,0)]*number_keypoints
sum_Z1 = [(0,0)]*number_keypoints
# it is to count the number of values available
n_HC = np.zeros((number_keypoints))
n_N1 = np.zeros((number_keypoints))
n_Z1 = np.zeros((number_keypoints))
for i in range(10):
    reprojection_error = reprojection_2D(gt1[i], gt2[i], gt3[i], removing_distance_points_3D[i], I_HC, I_N1, I_Z1, E_HC, E_HCN1, E_HCZ1, k_HC, k_N1, k_Z1, len(gt1[i]))
    sum_HC, n_HC = sum_error(sum_HC,reprojection_error[0], n_HC)
    sum_N1, n_N1 = sum_error(sum_N1,reprojection_error[1], n_N1)
    sum_Z1, n_Z1 = sum_error(sum_Z1,reprojection_error[2], n_Z1)
print("Removing highest distance and take mean")
mean_error_distance = error_printing(dividing_error(sum_HC,n_HC), dividing_error(sum_N1,n_N1), dividing_error(sum_Z1,n_Z1))
final_mean = 0
c = 0
for i in range (number_keypoints):
    if mean_error_distance[i][3] != 0:
        final_mean += mean_error_distance[i][3]
        c += 1
print("\nFinal mean: {:.2f}".format(final_mean/c))
print("\nNumber of points: \n HC: {} \n N1: {} \n Z1: {}".format(n_HC, n_N1, n_Z1))

Removing highest distance and take mean
Point 0: [0.42,4.76] ; [2.98,1.30] ; [4.94,10.69] ---- Mean: 4.18
Point 1: [0.09,7.55] ; [5.74,5.01] ; [7.97,10.38] ---- Mean: 6.12
Point 2: [0.43,0.47] ; [5.38,5.16] ; [4.88,4.87] ---- Mean: 3.53
Point 3: [1.85,13.22] ; [8.99,11.03] ; [15.89,32.82] ---- Mean: 13.97
Point 4: [8.15,17.97] ; [5.50,5.30] ; [5.98,13.14] ---- Mean: 9.34
Point 5: [1.13,17.77] ; [0.83,13.25] ; [5.06,5.56] ---- Mean: 7.27
Point 6: [0.94,2.53] ; [9.70,10.87] ; [9.44,14.10] ---- Mean: 7.93
Point 7: [0.60,4.21] ; [16.99,5.92] ; [15.57,11.65] ---- Mean: 9.16
Point 8: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 9: [20.28,12.92] ; [0.00,0.00] ; [4.96,12.51] ---- Mean: 12.67
Point 10: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 11: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 12: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 13: [0.00,0.00] ; [0.00,0.00] ; [0.00,0.00] ---- Mean: 0.00
Point 14: [0.00,0.00] ; [0